# Finetuning corpus embeddings using NUDGE
[NUDGE](https://www.arxiv.org/abs/2409.02343) is a novel simple and lightweight fine-tuning method that boosts accuracy when retrieving text using semantic similarity with pre-trained embedding models. NUDGE directly modifies the embeddings of data records to maximize the similarity between training queries and their ground-truth answers. NUDGE does so non-parametrically. Non-parametric means that NUDGE does not modify model parameters to generate better embeddings, as fine-tuning the embedding model, or training adaptors would. Instead, NUDGE directly changes the embeddings themselves. Compared with fine-tuning the pre-trained model and training adaptors, NUDGE provides 3.3x and 4.3x higher increase in accuracy and runs 200x and 3x faster, respectively. [Here](https://data-people-group.github.io/blogs/2024/09/05/nudge/) is a blog post on NUDGE, and [here](https://www.arxiv.org/abs/2409.02343) is the paper with more details.

We demonstrate NUDGE's effectiveness on a commonly used Information Retrieval benchmark called Scifact.

## Load the scifact benchmark

In [1]:
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from datasets import load_dataset

def load_hf_dataset(dataset_name):
    hf_dataset_name = f"sepz/{dataset_name}_ft"
    corpus = load_dataset(hf_dataset_name, "data_records", split="train")

    queries_train = load_dataset(hf_dataset_name, "qs", split="train")
    queries_validation = load_dataset(hf_dataset_name, "qs", split="dev")
    queries_test = load_dataset(hf_dataset_name, "qs", split="test")

    qrels_train = load_dataset(hf_dataset_name, "qs_rel", split="train")
    qrels_validation = load_dataset(hf_dataset_name, "qs_rel", split="dev")
    qrels_test = load_dataset(hf_dataset_name, "qs_rel", split="test")

    corpus = {
        str(corpus[i]["record_id"]): corpus[i]["text"] for i in range(len(corpus))
    }

    queries_train = {
        str(queries_train[i]["q_id"]): queries_train[i]["input"]
        for i in range(len(queries_train))
    }
    queries_validation = {str(r["q_id"]): r["input"] for r in queries_validation}
    queries_test = {str(r["q_id"]): r["input"] for r in queries_test}

    qrels_train = (
        qrels_train.to_pandas().groupby("q_id")["record_id"].apply(list).to_dict()
    )
    qrels_validation = (
        qrels_validation.to_pandas().groupby("q_id")["record_id"].apply(list).to_dict()
    )
    qrels_test = (
        qrels_test.to_pandas().groupby("q_id")["record_id"].apply(list).to_dict()
    )
    # convert to strings
    qrels_train = {str(k): [str(i) for i in v] for k, v in qrels_train.items()}
    qrels_validation = {
        str(k): [str(i) for i in v] for k, v in qrels_validation.items()
    }
    qrels_test = {str(k): [str(i) for i in v] for k, v in qrels_test.items()}

    # Load the dataset
    train_dataset = EmbeddingQAFinetuneDataset(
        corpus=corpus, queries=queries_train, relevant_docs=qrels_train
    )
    validation_dataset = EmbeddingQAFinetuneDataset(
        corpus=corpus, queries=queries_validation, relevant_docs=qrels_validation
    )
    test_dataset = EmbeddingQAFinetuneDataset(
        corpus=corpus, queries=queries_test, relevant_docs=qrels_test
    )

    return train_dataset, validation_dataset, test_dataset

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


INFO:datasets:PyTorch version 2.4.0 available.
PyTorch version 2.4.0 available.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the dataset and base embedding model

In [2]:
from llama_index.core.embeddings import resolve_embed_model

train_dataset, val_dataset, test_dataset = load_hf_dataset('scifact')
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


## Evaluation
A common Information Retrieval metric to report during evaluation is NDCG@k.

In [3]:
from typing import Optional

import torch
import numpy as np
from tqdm import tqdm
from llama_index.core.schema import TextNode
from llama_index.core.base.embeddings.base import BaseEmbedding
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.core import VectorStoreIndex

def build_retriever(
    dataset: EmbeddingQAFinetuneDataset,
    embed_model: BaseEmbedding | str,
    corpus_embeddings: Optional[torch.Tensor] = None,
    k: int=10
) -> BaseRetriever:
    corpus = dataset.corpus

    nodes = []
    for i, (id_, text) in enumerate(corpus.items()):
        if corpus_embeddings is not None:
            nodes.append(TextNode(id_=id_, text=text, embedding=corpus_embeddings[i].tolist()))
        else:
            nodes.append(TextNode(id_=id_, text=text))

    index = VectorStoreIndex(
        nodes=nodes, embeddings=corpus_embeddings, embed_model=embed_model, show_progress=True
    )
    return index.as_retriever(similarity_top_k=k)

def ndcg_at_k(
    dataset: EmbeddingQAFinetuneDataset,
    retriever: BaseRetriever,
    k: int=10
):
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    ndcg_scores = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_ids = relevant_docs[query_id]

        # Calculate NDCG
        ideal_dcg = np.sum([1/np.log2(i+2) for i in range(min(k, len(expected_ids)))])
        rel_scores = np.zeros(k)
        for j in range(min(k, len(retrieved_ids))):
            if retrieved_ids[j] in expected_ids:
                rel_scores[j] = 1
        dcg = np.sum([rel_scores[i]/np.log2(i+2) for i in range(len(rel_scores))])
        ndcg = dcg/ideal_dcg if ideal_dcg > 0 else 0
        
        ndcg_scores.append(ndcg)

    mean_ndcg = np.mean(ndcg_scores)
    return mean_ndcg

## Get the corpus embedding finetuning results
Next we use, [NUDGE](https://www.arxiv.org/abs/2409.02343), the state of the art method for finetuning corpus embeddings to maximize the accuracy of k-NN retrieval. We then take our new corpus embeddings along with the original embedding model to build a retriever. NUDGE only finetunes the corpus embeddings and does not change any of the parameters in the base embedding model. 

In [4]:
%%capture
from llama_index.experimental import Nudge

k = 10

nudge = Nudge(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    embed_model=base_embed_model,
    epochs=10000,
    train_batch_size=len(train_dataset.queries),
    val_batch_size=len(val_dataset.queries)
)
nudge.finetune()
nudge_corpus_embeddings = nudge.get_finetuned_corpus_embeddings()
nudge_retriever = build_retriever(train_dataset, base_embed_model, nudge_corpus_embeddings, k=k)
nudge_ndcg_test = ndcg_at_k(test_dataset, nudge_retriever, k)

INFO:llama_index.experimental.nudge.base:Use pytorch device: cuda
Use pytorch device: cuda


## Get the adapter finetuning results
We use a smaller batchsize than NUDGE above due to the adapter finetune baseline having a significantly slower training process. We also note that even with a batchsize the size of the dataset and 10k epochs the adapter finetuned model performs similarly to the hyperparams currently used.

In [5]:
%%capture
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine

embedding_adapater_finetune_engine = EmbeddingAdapterFinetuneEngine(
    train_dataset,
    base_embed_model,
    epochs=4,
    batch_size=10,
)
embedding_adapater_finetune_engine.finetune()
embedding_adapter_model = embedding_adapater_finetune_engine.get_finetuned_model()
ft_retriever = build_retriever(train_dataset, embedding_adapter_model, k=k)
ft_ndcg_test = ndcg_at_k(test_dataset, ft_retriever, k)

INFO:llama_index.finetuning.embeddings.adapter:Use pytorch device: cuda
Use pytorch device: cuda
INFO:llama_index.embeddings.adapter.base:Use pytorch device: cuda
Use pytorch device: cuda


## Get the baseline results

In [6]:
%%capture

base_retriever = build_retriever(train_dataset, base_embed_model, k=k)
bge_ndcg_test = ndcg_at_k(test_dataset, base_retriever, k)

## Display the results

In [7]:
print(f"bge test - ndcg@10: {bge_ndcg_test:.2f}")
print(f"adaptor finetune test - ndcg@10: {ft_ndcg_test:.2f}")
print(f"NUDGE test - ndcg@10: {nudge_ndcg_test:.2f}")

bge test - ndcg@10: 0.71
adaptor finetune test - ndcg@10: 0.72
NUDGE test - ndcg@10: 0.83
